In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import pickle
%matplotlib inline

In [ ]:
train = pd.read_excel('/content/drive/My Drive/FLIPR_6/Train_dataset.xlsx')
train.head()

,City,State,Type,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors,Covid Cases
0,Mumbai,Maharashtra,M.C,12442373.0,11978450.0,878.0,23.0,32.0,MEDIUM,219.0,150.0,0.700440,10924403.0,159.0,4408916.0,163115
1,Delhi,Delhi,M.C,11007835.0,9879172.0,858.0,27.0,30.0,MEDIUM,215.0,196.0,0.920018,9444722.0,148.0,2379169.0,80188
2,Bangalore,Karnataka,MPUA,8436675.0,4301326.0,936.0,28.0,37.0,HIGH,212.0,102.0,0.097085,7896728.0,123.0,636502.0,141000
3,Hyderabad,Telangana,MPUA,6809970.0,3637483.0,930.0,23.0,31.0,MEDIUM,217.0,118.0,0.827744,6333272.0,110.0,126078.0,55123
4,Ahmedabad,Gujarat,MPUA,5570585.0,3520085.0,852.0,29.0,25.0,LOW,227.0,109.0,0.847941,4746138.0,73.0,284973.0,33204


In [ ]:
test = pd.read_excel('/content/drive/My Drive/FLIPR_6/Test_dataset.xlsx')
test.head()

,City,State,Type,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
0,Tuensang,Nagaland,T.C,36774.0,NaN,931.0,23.0,10.0,MEDIUM,94.0,114.0,0.253390,34237.0,17.0,2769.0
1,Lakshmeshwar,Karnataka,T.M.C,36754.0,NaN,934.0,25.0,38.0,HIGH,62.0,160.0,0.192555,34328.0,13.0,636502.0
2,Zira,Punjab,M.Cl.,36732.0,NaN,883.0,29.0,35.0,HIGH,63.0,105.0,0.887882,32434.0,17.0,242367.0
3,Yawal,Maharashtra,M.Cl,36706.0,NaN,887.0,26.0,31.0,HIGH,60.0,174.0,0.407838,32558.0,11.0,4408916.0
4,Thana Bhawan,Uttar Pradesh,N.P.,36669.0,NaN,877.0,28.0,39.0,LOW,92.0,153.0,0.324456,32159.0,23.0,3104060.0


In [ ]:
df = pd.concat([train, test], axis=0, sort=True)
df.shape

(1288, 16)

In [ ]:
df['State'] = df['State'].replace(['Maharashtra', 'Gujarat', 'Andaman and Nicobar Islands', 'Goa', 'Dadra and Nagar Haveli', 'West Bengal'], 'West')
df['State'] = df['State'].replace(['Delhi', 'Rajasthan', 'Uttar Pradesh', 'Punjab', 'Haryana', 'Jammu and Kashmir', 'Chandigarh', 'Uttarakhand', 'Himachal Pradesh' ], 'North')
df['State'] = df['State'].replace(['Karnataka', 'Telangana', 'Tamil Nadu', 'Tamil nadu', 'Kerala','Puducherry'], 'South')
df['State'] = df['State'].replace(['Bihar', 'Jharkhand', 'Assam', 'Odisha', 'Tripura', 'Mizoram', 'Manipur','Sikkim', 'Nagaland', 'Meghalaya', 'Arunachal Pradesh', 'Andhra Pradesh', 'Andhra pradesh'], 'East')
df['State'] = df['State'].replace(['Madhya Pradesh', 'Andhra Pradesh', 'Chhattisgarh'], 'Central')

In [ ]:
categorical = ['State']

for var in categorical:
    df = pd.concat([df, 
                    pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [ ]:
df.head()

,# of hospitals,Avg Temp,City,Covid Cases,Female Population,Foreign Visitors,H Index,Median Age,Popuation [2001],Population [2011],SWM,Sex Ratio,Toilets Avl,Type,Water Purity,State_Central,State_East,State_North,State_South,State_West
0,159.0,32.0,Mumbai,163115.0,10924403.0,4408916.0,0.700440,23.0,11978450.0,12442373.0,MEDIUM,878.0,219.0,M.C,150.0,0,0,0,0,1
1,148.0,30.0,Delhi,80188.0,9444722.0,2379169.0,0.920018,27.0,9879172.0,11007835.0,MEDIUM,858.0,215.0,M.C,196.0,0,0,1,0,0
2,123.0,37.0,Bangalore,141000.0,7896728.0,636502.0,0.097085,28.0,4301326.0,8436675.0,HIGH,936.0,212.0,MPUA,102.0,0,0,0,1,0
3,110.0,31.0,Hyderabad,55123.0,6333272.0,126078.0,0.827744,23.0,3637483.0,6809970.0,MEDIUM,930.0,217.0,MPUA,118.0,0,0,0,1,0
4,73.0,25.0,Ahmedabad,33204.0,4746138.0,284973.0,0.847941,29.0,3520085.0,5570585.0,LOW,852.0,227.0,MPUA,109.0,0,0,0,0,1


In [ ]:
df.head()

,# of hospitals,Avg Temp,City,Covid Cases,Female Population,Foreign Visitors,H Index,Median Age,Popuation [2001],Population [2011],SWM,Sex Ratio,Toilets Avl,Type,Water Purity,State_high,State_low
0,159.0,32.0,Mumbai,163115.0,10924403.0,4408916.0,0.700440,23.0,11978450.0,12442373.0,MEDIUM,878.0,219.0,M.C,150.0,1,0
1,148.0,30.0,Delhi,80188.0,9444722.0,2379169.0,0.920018,27.0,9879172.0,11007835.0,MEDIUM,858.0,215.0,M.C,196.0,1,0
2,123.0,37.0,Bangalore,141000.0,7896728.0,636502.0,0.097085,28.0,4301326.0,8436675.0,HIGH,936.0,212.0,MPUA,102.0,1,0
3,110.0,31.0,Hyderabad,55123.0,6333272.0,126078.0,0.827744,23.0,3637483.0,6809970.0,MEDIUM,930.0,217.0,MPUA,118.0,0,1
4,73.0,25.0,Ahmedabad,33204.0,4746138.0,284973.0,0.847941,29.0,3520085.0,5570585.0,LOW,852.0,227.0,MPUA,109.0,0,1


In [ ]:
df.drop(['City', 'Popuation [2001]', 'Type'], axis = 'columns', inplace = True)

In [ ]:
var = ['Population [2011]','Sex Ratio','Median Age','Avg Temp', 'Toilets Avl', 'Water Purity', 'H Index', 'Female Population', '# of hospitals', 'Foreign Visitors']

for i in range(len(var)):
  df.replace(np.NAN, {var[i]: df[var[i]].median()}, inplace = True)
  

In [ ]:
df.isna().sum()

# of hospitals         0
Avg Temp               0
Covid Cases          501
Female Population      0
Foreign Visitors       0
H Index                0
Median Age             0
Population [2011]      0
SWM                   13
Sex Ratio              0
Toilets Avl            0
Water Purity           0
State_Central          0
State_East             0
State_North            0
State_South            0
State_West             0
dtype: int64

In [ ]:
df['SWM'].fillna('LOW', inplace = True)

In [ ]:

categorical = ['SWM']

for var in categorical:
    df = pd.concat([df, 
                    pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [ ]:
from sklearn.preprocessing import StandardScaler
continuous = ['Population [2011]','Sex Ratio','Median Age','Avg Temp', 'Toilets Avl', 'Water Purity', 'H Index', 'Female Population', '# of hospitals', 'Foreign Visitors']

scaler = StandardScaler()

for var in continuous:
    df[var] = df[var].astype('float64')
    df[var] = scaler.fit_transform(df[var].values.reshape(-1, 1))

In [ ]:
df.isna().sum()

# of hospitals         0
Avg Temp               0
Covid Cases          501
Female Population      0
Foreign Visitors       0
H Index                0
Median Age             0
Population [2011]      0
Sex Ratio              0
Toilets Avl            0
Water Purity           0
State_Central          0
State_East             0
State_North            0
State_South            0
State_West             0
SWM_HIGH               0
SWM_LOW                0
SWM_MEDIUM             0
dtype: int64

In [ ]:
df.shape

(1288, 19)

In [ ]:
X_train = df[pd.notnull(df['Covid Cases'])]

In [ ]:
from scipy import stats

#outlier detection and removal
z = np.abs(stats.zscore(X_train))
print(z)

[[ 4.06538597  0.15347807 10.36522443 ...  0.72674297  0.71047879
   1.46132069]
 [ 3.68482751  0.13687587  4.87282495 ...  0.72674297  0.71047879
   1.46132069]
 [ 2.81992192  0.87936291  8.90050951 ...  1.37600231  0.71047879
   0.68431249]
 ...
 [ 0.50131555  0.42722981  0.2493391  ...  1.37600231  0.71047879
   0.68431249]
 [ 0.91647023  3.62112314  0.2952377  ...  0.72674297  0.71047879
   1.46132069]
 [ 0.39752687  0.87936291  0.29113133 ...  1.37600231  0.71047879
   0.68431249]]


In [ ]:
X_train=X_train[(z < 7).all(axis=1)]
X_train.shape

(779, 19)

In [ ]:
##train test split
#X_train = df[pd.notnull(df['Covid Cases'])].drop(['Covid Cases'], axis=1)
y_train = X_train[pd.notnull(X_train['Covid Cases'])]['Covid Cases']
X_test = df[pd.isnull(df['Covid Cases'])].drop(['Covid Cases'], axis=1)

In [ ]:
X_train.drop('Covid Cases', axis = 'columns', inplace = True)

In [ ]:
print('training data: ', X_train.shape)
print('testing data: ', X_test.shape)
print('train label: ', y_train.shape)

training data:  (779, 18)
testing data:  (501, 18)
train label:  (779,)


In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.2, random_state=50)

In [ ]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

train_x.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in train_x.columns.values]

test_x.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in test_x.columns.values]

In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
model_xgb.fit(train_x, train_y)

[12:11:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
y_xgb_pred = model_xgb.predict(test_x)

In [ ]:
from sklearn import metrics
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(np.log1p(test_y), np.log1p(y_xgb_pred))))

Root Mean Squared Error: 0.4655424362718352


In [ ]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]

In [ ]:
y_pred = model_xgb.predict(X_test)

In [ ]:
result = pd.concat([test['City'], pd.DataFrame(y_pred)], axis=1)

In [ ]:
result.rename(columns={0:"Covid Cases"}, inplace = True)

In [ ]:
result.head()

,City,Covid Cases
0,Tuensang,2414.376221
1,Lakshmeshwar,2390.112549
2,Zira,2264.909424
3,Yawal,2405.205078
4,Thana Bhawan,2513.395508


In [ ]:
result.to_csv(r'task1_result.csv', index = False)

In [ ]:
#save model
import pickle
filename = "Task1_model.pickle"
pickle.dump(model_xgb, open(filename, 'wb'))

In [ ]:
from sklearn.svm import SVR
model_svr = SVR(C=10,kernel='linear')
model_svr.fit(train_x, train_y)

SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
y_svr_pred = model_svr.predict(test_x)

In [ ]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(np.log1p(test_y), np.log1p(y_svr_pred))))

Root Mean Squared Error: 0.5598042480849788
